In [ ]:
# !pip install pythreejs
# !pip install numpy, scipy
# !pip install notebook==6.5.4
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user

In [ ]:
from pythreejs import * 
from IPython.display import display
import ipywidgets as widgets
import numpy as np
import time

In [ ]:
class Particle:
    def __init__(self, pos, vel, mass):
        self.position = pos
        self.velocity = vel
        self.force = np.array([0, 0, 0])
        self.mass = mass
        
class Particle_System:
    def __init__(self, time_step=0.033):
        
        self.particles = []
        
        # Types of forces
        # 1. Constant e.g. gravity
        # 2. position dependent e.g. forces fields, winds
        # 3. velocity dependent e.g. drag, friction
        # 4. n-ary e.g. springs
        # 5. collision
        self.forces = []
        
        self.time_step = 0.01
        self.gravity = np.array([0, -9.8, 0])
    
    def add_particle(self, particle):
        if isinstance(particle, list):
            self.particles.extend(particle)
            
        elif isinstance(particle, Particle):       
            self.particles.append(particle)
        
        else:
            raise ValueError("Invalid particle type")
        
    def add_force(self, force):
        if isinstance(force, list):
            self.forces.extend(force)
            
        elif isinstance(force, Force):       
            self.forces.append(force)
        
        else:
            raise ValueError("Invalid force type")
        
        
    def simulate(self, steps=100):
        for i in range(steps):
            for force in self.forces:
                force.apply(self.particles)
            
            for p in self.particles:
                p.solve(self.time_step)
            
            self.render()
            time.sleep(self.time_step)
    
    def render(self,):
        pass

    
class Force:
    def apply(self, particles):
        pass

class Gravity(Force):
    def __init__(self, g=np.array([0, -9.8, 0])):
        self.g = g
        
    def apply(self, particles):
        for p in particles:
            p.force += p.mass * self.g

class Drag(Force):
    def __init__(self, c=0.1):
        self.c = c
        
    def apply(self, particles):
        for p in particles:
            p.force += -self.c * p.velocity